# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([31])

INITIALIZATION_METHOD = 'Amplitude'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 1, 2, 3]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=3)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(3), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(3), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[0, 1])

    # Second layer
    ArbitraryUnitary(params[15:30], wires=[1, 2])

    # Third layer
    qml.RY(params[30], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254]

Example features: [-2.80374714 -1.29635958 -0.34733335 -0.55602548 -1.29805526 -0.27010804
  0.87562563 -0.63683318]

Expectation value: -0.41682031676780973

 0: ──╭QubitStateVector(M0)──H──RZ(0.65)──H──RX(1.57)──RZ(0.948)──RX(-1.57)──RZ(0.388)──╭RZ(0.641)──H──────────────────────╭RZ(0.127)──H──RX(1.57)──╭RZ(0.239)──RX(-1.57)──RX(1.57)──╭RZ(0.254)──RX(-1.57)────────────╭RZ(0.426)──H────────────────────────────────────────────────────╭RZ(0.745)──H──────────H──╭RZ(0.381)──H──RX(1.57)──╭RZ(0.262)──RX(-1.57)──╭RZ(0.0533)─────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5358943577430972
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.6744297719087635
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.7253301320528212
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.7046818727490997
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7344537815126051
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.7243697478991596
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.7368547418967587
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.756782713085234
best accuracy so far!
epoch 71

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7531812725090036
test_accuracies: [0.7368547418967587, 0.756782713085234, 0.7553421368547419, 0.7476590636254502, 0.7498199279711885, 0.7404561824729892, 0.7563025210084033, 0.7495798319327731, 0.7603841536614646, 0.765906362545018, 0.7649459783913566, 0.7476590636254502, 0.7404561824729892, 0.7517406962785114, 0.7527010804321729, 0.7397358943577431, 0.7382953181272509, 0.7534213685474189, 0.7596638655462185, 0.758703481392557, 0.763265306122449, 0.7603841536614646, 0.7594237695078031, 0.7639855942376951, 0.7649459783913566, 0.7577430972388955, 0.7522208883553422, 0.7711884753901561, 0.7582232893157264, 0.7531812725090036]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7546218487394958
test_accuracies: [0.756782713085234, 0.7553421368547419, 0.7476590636254502, 0.7498199279711885, 0.7404561824729892, 0.756302521

epoch 452
epoch 453
epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.7570228091236495
test_accuracies: [0.7476590636254502, 0.7404561824729892, 0.7517406962785114, 0.7527010804321729, 0.7397358943577431, 0.7382953181272509, 0.7534213685474189, 0.7596638655462185, 0.758703481392557, 0.763265306122449, 0.7603841536614646, 0.7594237695078031, 0.7639855942376951, 0.7649459783913566, 0.7577430972388955, 0.7522208883553422, 0.7711884753901561, 0.7582232893157264, 0.7531812725090036, 0.7546218487394958, 0.7438175270108043, 0.7330132052821128, 0.7642256902761104, 0.7375750300120048, 0.7500600240096038, 0.7512605042016807, 0.7522208883553422, 0.7639855942376951, 0.7630252100840336, 0.7570228091236495]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7433373349339736
test_accuracies: [0.7404561824729892, 0.7517406962785114, 0.7527010804321729, 0.7397358943577431, 0.7382953181272509, 0.7534213685474189

epoch 564
epoch 565
epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.7579831932773109
test_accuracies: [0.7594237695078031, 0.7639855942376951, 0.7649459783913566, 0.7577430972388955, 0.7522208883553422, 0.7711884753901561, 0.7582232893157264, 0.7531812725090036, 0.7546218487394958, 0.7438175270108043, 0.7330132052821128, 0.7642256902761104, 0.7375750300120048, 0.7500600240096038, 0.7512605042016807, 0.7522208883553422, 0.7639855942376951, 0.7630252100840336, 0.7570228091236495, 0.7433373349339736, 0.7683073229291717, 0.7673469387755102, 0.77046818727491, 0.7452581032412966, 0.7635054021608644, 0.7615846338535415, 0.7618247298919568, 0.7505402160864345, 0.7599039615846339, 0.7579831932773109]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.7505402160864345
test_accuracies: [0.7639855942376951, 0.7649459783913566, 0.7577430972388955, 0.7522208883553422, 0.7711884753901561, 0.7582232893157264, 0.7531812725090036

epoch 675
epoch 676
epoch 677
epoch 678
epoch 679
epoch 680
accuracy: 0.7745498199279712
test_accuracies: [0.7642256902761104, 0.7375750300120048, 0.7500600240096038, 0.7512605042016807, 0.7522208883553422, 0.7639855942376951, 0.7630252100840336, 0.7570228091236495, 0.7433373349339736, 0.7683073229291717, 0.7673469387755102, 0.77046818727491, 0.7452581032412966, 0.7635054021608644, 0.7615846338535415, 0.7618247298919568, 0.7505402160864345, 0.7599039615846339, 0.7579831932773109, 0.7505402160864345, 0.756782713085234, 0.75078031212485, 0.7788715486194477, 0.7591836734693878, 0.7555822328931573, 0.7711884753901561, 0.7430972388955582, 0.761344537815126, 0.7711884753901561, 0.7745498199279712]
epoch 681
epoch 682
epoch 683
epoch 684
epoch 685
epoch 686
epoch 687
epoch 688
epoch 689
epoch 690
accuracy: 0.756062424969988
test_accuracies: [0.7375750300120048, 0.7500600240096038, 0.7512605042016807, 0.7522208883553422, 0.7639855942376951, 0.7630252100840336, 0.7570228091236495, 0.74333733493

epoch 789
epoch 790
accuracy: 0.7611044417767107
test_accuracies: [0.77046818727491, 0.7452581032412966, 0.7635054021608644, 0.7615846338535415, 0.7618247298919568, 0.7505402160864345, 0.7599039615846339, 0.7579831932773109, 0.7505402160864345, 0.756782713085234, 0.75078031212485, 0.7788715486194477, 0.7591836734693878, 0.7555822328931573, 0.7711884753901561, 0.7430972388955582, 0.761344537815126, 0.7711884753901561, 0.7745498199279712, 0.756062424969988, 0.7798319327731092, 0.7637454981992797, 0.7663865546218487, 0.7450180072028811, 0.7764705882352941, 0.7663865546218487, 0.7709483793517407, 0.7380552220888356, 0.7745498199279712, 0.7611044417767107]
epoch 791
epoch 792
epoch 793
epoch 794
epoch 795
epoch 796
epoch 797
epoch 798
epoch 799
epoch 800
accuracy: 0.7490996398559424
test_accuracies: [0.7452581032412966, 0.7635054021608644, 0.7615846338535415, 0.7618247298919568, 0.7505402160864345, 0.7599039615846339, 0.7579831932773109, 0.7505402160864345, 0.756782713085234, 0.750780312124

epoch 901
epoch 902
epoch 903
epoch 904
epoch 905
epoch 906
epoch 907
epoch 908
epoch 909
epoch 910
accuracy: 0.7553421368547419
test_accuracies: [0.7591836734693878, 0.7555822328931573, 0.7711884753901561, 0.7430972388955582, 0.761344537815126, 0.7711884753901561, 0.7745498199279712, 0.756062424969988, 0.7798319327731092, 0.7637454981992797, 0.7663865546218487, 0.7450180072028811, 0.7764705882352941, 0.7663865546218487, 0.7709483793517407, 0.7380552220888356, 0.7745498199279712, 0.7611044417767107, 0.7490996398559424, 0.7548619447779111, 0.7611044417767107, 0.7714285714285715, 0.7791116446578632, 0.7786314525810324, 0.7671068427370948, 0.7642256902761104, 0.7690276110444177, 0.7774309723889556, 0.77046818727491, 0.7553421368547419]
epoch 911
epoch 912
epoch 913
epoch 914
epoch 915
epoch 916
epoch 917
epoch 918
epoch 919
epoch 920
accuracy: 0.7690276110444177
test_accuracies: [0.7555822328931573, 0.7711884753901561, 0.7430972388955582, 0.761344537815126, 0.7711884753901561, 0.774549819

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [-0.02296152  1.15483185  0.4166237   1.28235643 -0.06535592 -0.71282209
  0.91035007 -0.15252492  0.42616295  0.20882011  1.57839705  1.69318018
 -0.22703855  0.83181662  0.5687541  -0.0374327   0.12525712  0.13205984
 -0.13151437 -0.36572298  2.06196337  0.86582585  0.40657559  0.19192114
  0.4088203   0.51570916  1.29206236  0.73700922  1.36297351  0.06555319
  0.07466066]


# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7857142857142857


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 12.615741729736328
training time: 9869.861811161041
test time: 226.28074169158936
total time: 10108.758294582367
